<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Creating-dropdowns" data-toc-modified-id="Creating-dropdowns-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creating dropdowns</a></span></li><li><span><a href="#Putting-it-all-together" data-toc-modified-id="Putting-it-all-together-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Putting it all together</a></span></li><li><span><a href="#Another-example" data-toc-modified-id="Another-example-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Another example</a></span></li><li><span><a href="#Multiple-figures" data-toc-modified-id="Multiple-figures-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Multiple figures</a></span></li><li><span><a href="#Deployment" data-toc-modified-id="Deployment-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Deployment</a></span></li></ul></div>

<img src=https://www.pngkit.com/png/full/861-8618685_numfocus-plotly-dash-logo.png alt="logo" width="400"/>

# Introduction

Dash is a library for creating interactive dashboards in python. The [documentation](https://dash.plotly.com/) is excellent, but examples can be a little verbose and hard to read. It is also designed to run as a server and to make the dashboards available via a web browser.

First we will import dash package, dash_html_components for HTML classes, dash_core_components for elements such as graph, dropdown etc, and plotly packages for creating plots.

In [1]:
! pip install dash


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objects as go
import plotly.express as px

/var/folders/dk/95f115l12ms7fs6v815_g44c0000gn/T/ipykernel_41931/1432543730.py:2: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/var/folders/dk/95f115l12ms7fs6v815_g44c0000gn/T/ipykernel_41931/1432543730.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In the below code we are initializing our dash app using dash package. Then, we are reading the stock prices data for different companies from 2018 to 2019. We are creating stock_prices function that returns the line chart for Google’s stock prices.

In [3]:
app = dash.Dash()   #initialising dash app
df = px.data.stocks() #reading stock price dataset 

def stock_prices():
    # Function for creating line chart showing Google stock prices over time 
    fig = go.Figure([go.Scatter(x = df['date'], y = df['GOOG'],\
                     line = dict(color = 'firebrick', width = 4), name = 'Google')
                     ])
    fig.update_layout(title = 'Prices over time',
                      xaxis_title = 'Dates',
                      yaxis_title = 'Prices'
                      )
    return fig  

 
app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Styling using html components', style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40}),

        
        dcc.Graph(id = 'line_plot', figure = stock_prices())    
    ]
                     )

* At line 16, we are setting our layout using html Div component, which is a wrapper within which the elements(heading, graph) of the layout will be created. The Div component contains arguments such as id (a unique identifier of the element), style (for setting the width, height, color etc) and children (equal to square bracket within which elements of the layout are initialised).

* Inside the children component (of html.Div) we are creating html H1 heading at line 17 using H1 function. Inside the function, we are setting the unique id of the function (id = ‘H1’), children property using which we set the text of the heading and style property as a dictionary within which we are setting styling such as center aligning the text, setting top and bottom margin to 40 pixel. 

* At line 21, we are using dash core component (dcc) to create graph , where we are setting the id of the graph and the figure argument, which is equal to the function call (stock_pricest()) that returns the plotly figure object.

In order to view our application, we need to run our web server

On running the app, you will see that the app is running on http://127.0.0.1:8050/ , which is your local server. Copy this url and paste it in your browser and you will see the below vizualisation.

In [5]:
if __name__ == '__main__':
    app.run_server(host='127.0.0.1', port=8050)  # Example for localhost and port 8050

Dash is designed to create interactive dashboards, not just to render static plots and display them in Jupyter. Behing the scenes it uses Flask, a web application framework. This means we can (quite) easily get our dash apps online to share with others! More on this later.

We can also use HTML components to arrange and style our dashboards. Notice the heading - html.H1 should ring a bell for anyone who has done web dev.

# Creating dropdowns

Now, we will create a dropdown using dash core components. Using the dropdown, we will be able to select the stocks of Google, Apple or Amazon.

Dropdowns are created using the Dropdown() function, which has the following arguments -

* id — Unique identifier of the dropdown.
* options — Sets the ‘label’ (the text visible in the dropdown) and ‘value’ (used by dash to communicate with callbacks) as key value pair.
* value — default selection for the dropdown.

In [6]:
dcc.Dropdown( id = 'dropdown',
options = [
    {'label':'Google', 'value':'GOOG' },
    {'label': 'Apple', 'value':'AAPL'},
    {'label': 'Amazon', 'value':'AMZN'},
    ],
value = 'GOOGL'       
)

Dropdown(options=[{'label': 'Google', 'value': 'GOOG'}, {'label': 'Apple', 'value': 'AAPL'}, {'label': 'Amazon', 'value': 'AMZN'}], value='GOOGL', id='dropdown')

Next we need to make the callbacks interactive:
    
A callback is initialised using @app.callback(), which is followed by a function definition. Within this function, we define what happens on changing the value of the dropdown.    

Output: used to define the components within the layout that will be updated. In our case we watn to update stick prices

Input: used to define the components whose value will trigger the callback. The input is defined within a list

In [7]:
from dash.dependencies import Input, Output  

@app.callback(Output(component_id='line_plot', component_property= 'figure'),
              [Input(component_id='dropdown', component_property= 'value')])
def graph_update(dropdown_value):
    print(dropdown_value)
    fig = go.Figure([go.Scatter(x = df['date'], y = df['{}'.format(dropdown_value)],\
                     line = dict(color = 'firebrick', width = 4))
                     ])
    
    fig.update_layout(title = 'Stock prices over time',
                      xaxis_title = 'Dates',
                      yaxis_title = 'Prices'
                      )
    return fig  

# Putting it all together

In [10]:
import dash
import dash_html_components as html
import plotly.graph_objects as go
import dash_core_components as dcc
import plotly.express as px
from dash.dependencies import Input, Output


app = dash.Dash()

df = px.data.stocks()


app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Styling using html components', style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40}),

        dcc.Dropdown( id = 'dropdown',
        options = [
            {'label':'Google', 'value':'GOOG' },
            {'label': 'Apple', 'value':'AAPL'},
            {'label': 'Amazon', 'value':'AMZN'},
            ],
        value = 'GOOG'),
        dcc.Graph(id = 'bar_plot')
    ])
    
    
@app.callback(Output(component_id='bar_plot', component_property= 'figure'),
              [Input(component_id='dropdown', component_property= 'value')])
def graph_update(dropdown_value):
    print(dropdown_value)
    fig = go.Figure([go.Scatter(x = df['date'], y = df['{}'.format(dropdown_value)],\
                     line = dict(color = 'firebrick', width = 4))
                     ])
    
    fig.update_layout(title = 'Stock prices over time',
                      xaxis_title = 'Dates',
                      yaxis_title = 'Prices'
                      )
    return fig  



if __name__ == '__main__':
    app.run_server(host='127.0.0.1', port=8050)  # Example for localhost and port 8050

GOOG


# Another example

You can upload the diabetes.csv file by running the code below to read the data from a URL - a useful technique.

In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('https://raw.githubusercontent.com/surabhim/Diabetes/master/Diabetes.csv', skiprows=9,
                 names=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
                 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']) # or read data/diabetes.csv from Tuesday
print(df.shape)
df.head()

(768, 9)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


For some background, the columns definitions (from the original file):
1. Number of times pregnant
2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
3. Diastolic blood pressure (mm Hg)
4. Triceps skin fold thickness (mm)
5. 2-Hour serum insulin (mu U/ml)
6. Body mass index (weight in kg/(height in m)^2)
7. Diabetes pedigree function
8. Age (years)
9. Class variable (0 or 1)

Now, we could start exploring and plotting with seaborn, but that's not our goal today - let's see if we can make some interactive visuals!

In [14]:
app = dash.Dash()

app.layout = html.Div(id='parent', children=[
    html.H1(id='H1', children='Diabetes Dataset Visualization', style={'textAlign': 'center',
                                                                      'marginTop': 40, 'marginBottom': 40}),
    dcc.Dropdown(id='dropdown',
                 options=[
                     {'label': 'Pregnancies', 'value': 'Pregnancies'},
                     {'label': 'Glucose', 'value': 'Glucose'},
                     {'label': 'SkinThickness', 'value': 'SkinThickness'},
                     {'label': 'Insulin', 'value': 'Insulin'},
                     {'label': 'BMI', 'value': 'BMI'},
                     {'label': 'DiabetesPedigreeFunction', 'value': 'DiabetesPedigreeFunction'},
                     {'label': 'Age', 'value': 'Age'}
                 ],
                 value='Glucose'),
    dcc.Graph(id='bar_plot')
])


@app.callback(Output(component_id='bar_plot', component_property='figure'),
              [Input(component_id='dropdown', component_property='value')])
def graph_update(dropdown_value):
    print(dropdown_value)
    fig = go.Figure([go.Scatter(x=df[dropdown_value], y=df['BloodPressure'],
                                mode='markers', marker=dict(color='firebrick'))
                     ])
    
    fig.update_layout(title=f'{dropdown_value} vs BloodPressure',
                      xaxis_title=dropdown_value,
                      yaxis_title='BloodPressure')
    return fig

if __name__ == '__main__':
    app.run_server(host='127.0.0.1', port=8050)  # Example for localhost and port 8050


Glucose
Glucose
Insulin
Glucose


# Multiple figures
There is no reason we can't make multiple figures and arrange them however we like. Here's an example:

In [15]:
app = dash.Dash()

app.layout = html.Div(id='parent', children=[
    html.H1(id='H1', children='Diabetes Dataset Visualization', style={'textAlign': 'center',
                                                                      'marginTop': 40, 'marginBottom': 40}),
    dcc.Dropdown(id='dropdown',
                 options=[
                     {'label': 'Pregnancies', 'value': 'Pregnancies'},
                     {'label': 'Glucose', 'value': 'Glucose'},
                     {'label': 'SkinThickness', 'value': 'SkinThickness'},
                     {'label': 'Insulin', 'value': 'Insulin'},
                     {'label': 'BMI', 'value': 'BMI'},
                     {'label': 'DiabetesPedigreeFunction', 'value': 'DiabetesPedigreeFunction'},
                     {'label': 'Age', 'value': 'Age'}
                 ],
                 value='Glucose'),
    html.Div(children=[
        dcc.Graph(id='bp_plot', style={'display': 'inline-block', 'width': '48%'}),
        dcc.Graph(id='outcome_plot', style={'display': 'inline-block', 'width': '48%'})
    ])
])


@app.callback([Output(component_id='bp_plot', component_property='figure'),
               Output(component_id='outcome_plot', component_property='figure')],
              [Input(component_id='dropdown', component_property='value')])
def graph_update(dropdown_value):
    print(dropdown_value)
    
    bp_fig = go.Figure(data=go.Scatter(x=df[dropdown_value], y=df['BloodPressure'],
                                   mode='markers', marker=dict(color='firebrick')))
    bp_fig.update_layout(title=f'{dropdown_value} vs BloodPressure',
                         xaxis_title=dropdown_value,
                         yaxis_title='BloodPressure')
    
    outcome_fig = go.Figure(data=go.Scatter(x=df[dropdown_value], y=df['Outcome'],
                                        mode='markers', marker=dict(color='blue')))
    outcome_fig.update_layout(title=f'{dropdown_value} vs Outcome',
                              xaxis_title=dropdown_value,
                              yaxis_title='Outcome')
    
    return bp_fig, outcome_fig


if __name__ == '__main__':
    app.run_server(host='127.0.0.1', port=8050, debug=True)  # Example for localhost and port 8050


Glucose
Glucose
Glucose


# Deployment

Embedding a Dash app into a website requires hosting the Dash app on a server and then using an iframe to embed it. 

First, you need to host your Dash app on a public server. You can use platforms like Heroku, AWS, or any other web hosting service. 

As of November 28, 2022, Heroku has discontinued its free tier. However, you can still deploy Dash apps on other free hosting platforms like Render, Vercel, or Google Cloud Platform (with free tier usage). 
